---
### Universidad de Costa Rica
#### IE0405 - Modelos Probabilísticos de Señales y Sistemas
---

# `Py9` - *Documentación de código*

> Utilizando las reglas de los *docstrings* en Python, un proyecto con archivos bien comentados pueden convertirse en una **documentación** disponible en distintos formatos de presentación (PDF, HTML, ePub, entre otros) que es útil para los usuarios del software y para la presentación de resultados en contextos de desarrollo académico o profesional.

---

## Introducción

### *Docstrings*

Un [*docstring*](https://www.python.org/dev/peps/pep-0257/) es una cadena de caracteres que viene como primera información en la definición de clases, métodos, módulos o funciones.

> Este *docstring* se convierte en el atributo especial `__doc__` de ese objeto.

Está acotado por la notación de triple comillas:

```python
def foo():
    """
    Este es un comentario que funciones como docstring.
    """
    (...)
```

### reStructuredText

El estándar **reStructuredText** es utilizado para dar formato al texto.

### Referencia

Usar este artículo como referencia (o cualquier otro): https://realpython.com/documenting-python-code/

---
## 9.1 - Sphinx

Aun sin conocer una expresión para la función de densidad $f_Y(y)$ de una variable aleatoria $Y$ producto de una transformación $Y = T(X)$, es posible conocer el efecto que tiene esta transformación sobre una muestra de datos aleatorios con una distribución de probabilidad $X$ conocida.

In [ ]:
import numpy as np

---
## 8.2 - Ejemplo del canal de comunicaciones AWGN con la modulación OOK

La modulación **OOK** (*on/off keying*, codificación por encendido y apagado) consiste en enviar una forma de onda sinusoidal para el bit 1, y no enviar nada para el bit 0. Esta es una modulación binaria de un bit por símbolo.

#### Generación de los bits

In [ ]:
import numpy as np

---
### Más información

* Función [scipy.integrate.trapz](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.trapz.html)
* Modulación [OOK](https://en.wikipedia.org/wiki/On%E2%80%93off_keying)
---

---

**Universidad de Costa Rica**

Facultad de Ingeniería

Escuela de Ingeniería Eléctrica

---